# Определение уровня лояльности клиентов (NPS) телекоммуникационной компании из России

**Заказчик** — телекоммуникационная компания, которая оказывает услуги на территории всего СНГ.

**Цель:** Определить текущий уровень потребительской лояльности, или NPS (от англ. Net Promoter Score), среди клиентов из России на основе результата опроса «Оцените по шкале от 1 до 10 вероятность того, что вы порекомендуете компанию друзьям и знакомым».

**Входные данные:** База данных в SQLite. 

Таблица `user`

* *user_id* -	Идентификатор клиента, первичный ключ таблицы
* *lt_day* - Количество дней «жизни» клиента
* *age* - Возраст клиента в годах
* *gender_segment* - Пол клиента (1 – женщина, 0 – мужчина)
* *os_name* - Тип операционной системы
* *cpe_type_name* - Тип устройства
* *location_id* - Идентификатор домашнего региона клиента, внешний ключ, отсылающий к таблице location
* *age_gr_id* - Идентификатор возрастного сегмента клиента, внешний ключ, отсылающий к таблице age_segment
* *tr_gr_id* - Идентификатор сегмента клиента по объёму потребляемого трафика в месяц, внешний ключ, отсылающий к таблице traffic_segment
* *lt_gr_id* - Идентификатор сегмента клиента по количеству месяцев «жизни», внешний ключ, отсылающий к таблице lifetime_segment
* *nps_score* - Оценка клиента в NPS-опросе (от 1 до 10)

Таблица `location`

* *location_id* - Идентификатор записи, первичный ключ
* *country* - Страна
* *city* - Город

Таблица `age_segment`

* *age_gr_id* -	Идентификатор сегмента, первичный ключ
* *bucket_min* - Минимальная граница сегмента
* *bucket_max* - Максимальная граница сегмента
* *title* - Название сегмента

Таблица `traffic_segment`

* *tr_gr_id* - Идентификатор сегмента, первичный ключ
* *bucket_min* - Минимальная граница сегмента
* *bucket_max* - Максимальная граница сегмента
* *title* - Название сегмента

Таблица `lifetime_segment`

* *lt_gr_id* - Идентификатор сегмента, первичный ключ
* *bucket_min* - Минимальная граница сегмента
* *bucket_max* - Максимальная граница сегмента
* *title* -	Название сегмента

In [1]:
# импорт библиотек
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

## Подключение к базе данных

In [2]:
path_to_db = '/datasets/telecomm_csi.db'
engine = create_engine(f'sqlite:///{path_to_db}', echo = False)

## Выгрузка необходимых данных

Выгрузим данные, объедив их в общую таблицу так, чтобы:
- по результатам опроса сформировать 3 группы:
  * 9-10 баллов — «cторонники» (англ. promoters);
  * 7-8 баллов — «нейтралы» (англ. passives);
  * 0-6 баллов — «критики» (англ. detractors).
- по лайфтайму 2 группы:
  * не более 365 дней - новые клиенты
  * более 365 дней - старые клиенты

In [3]:
query = """
SELECT u.user_id,
        u.lt_day,
        CASE
            WHEN u.lt_day > 365 THEN 'No'
            ELSE 'Yes'
        END AS is_new,
        u.age,
        CAST (u.gender_segment AS varchar) AS gender_segment,
        u.os_name,
        u.cpe_type_name,
        l.country,
        l.city,
        a.title AS age_segment,
        t.title AS traffic_segment,
        lt.title AS lifetime_segment,
        u.nps_score,
        CASE
            WHEN u.nps_score > 8 THEN 'сторонники'
            WHEN u.nps_score < 7 THEN 'критики'
            ELSE 'нейтралы'
        END AS nps_group
FROM user AS u
LEFT JOIN location AS l ON u.location_id=l.location_id
LEFT JOIN age_segment AS a ON u.age_gr_id=a.age_gr_id
LEFT JOIN traffic_segment AS t ON u.tr_gr_id=t.tr_gr_id
LEFT JOIN lifetime_segment AS lt ON u.lt_gr_id=lt.lt_gr_id
"""

Сохраним таблицу прочитанную в pandas в переменную df

In [4]:
df = pd.read_sql(query, engine)
df.head(3)

,user_id,lt_day,is_new,age,gender_segment,os_name,cpe_type_name,country,city,age_segment,traffic_segment,lifetime_segment,nps_score,nps_group
0,A001A2,2320,No,45.0,1.0,ANDROID,SMARTPHONE,Россия,Уфа,05 45-54,04 1-5,08 36+,10,сторонники
1,A001WF,2344,No,53.0,0.0,ANDROID,SMARTPHONE,Россия,Киров,05 45-54,04 1-5,08 36+,10,сторонники
2,A003Q7,467,No,57.0,0.0,ANDROID,SMARTPHONE,Россия,Москва,06 55-64,08 20-25,06 13-24,10,сторонники


Выгрузим таблицу в формате csv для построения дашбордов

In [5]:
df.to_csv('telecomm_csi_tableau.csv', index=False)

## Cсылка на дашборд на сайте Tableau Public:

https://public.tableau.com/authoring/telecommcopy/ageisnewgen#1

## Ссылка на pdf-файл с презентацией:

https://disk.yandex.ru/i/yOqkfkqRUK2wWQ